In [ ]:
!pip install transformers

## 허깅페이스 코드 붙혀넣기

허깅페이스에서 대표적인 Object Detection 모델을 찾고 코드 전체를 실행해봅니다.

링크: https://huggingface.co/facebook/detr-resnet-50

In [ ]:
# 허깅페이스에서 불러온 코드 붙여넣기


## (심화) 코드 이해하기

### 1. 필요한 라이브러리 임포트
먼저 필요한 라이브러리를 임포트합니다.

- DetrImageProcessor 및 DetrForObjectDetection: Hugging Face에서 제공하는 Object Detection 모델과 전처리 도구.
- torch: 딥러닝 프레임워크인 PyTorch.
- Image: 이미지 로드 및 처리.
- requests: URL로부터 이미지를 로드하기 위한 HTTP 요청.
- matplotlib.pyplot 및 matplotlib.patches: 이미지를 시각화하고 객체 감지 결과를 그리기 위한 도구.

In [ ]:
# 필요한 라이브러리 임포트
import torch
from transformers import DetrImageProcessor, DetrForObjectDetection  # Object Detection 모델 및 전처리 도구
from PIL import Image  # 이미지 처리
import requests  # 이미지 URL 요청
import matplotlib.pyplot as plt  # 시각화 도구
import matplotlib.patches as patches  # 박스 그리기 도구


### 2. 이미지 로드 및 모델 준비
이미지를 로드하고 객체 감지 모델을 초기화합니다. 이미지는 인터넷 URL 또는 로컬 경로에서 불러올 수 있습니다.

(참고) 함수설명

- url: 인터넷에서 이미지를 가져올 URL.
- Image.open: URL이나 로컬 파일 경로에서 이미지를 불러옴.
- DetrImageProcessor.from_pretrained: 사전 학습된 모델을 위한 전처리기.
- DetrForObjectDetection.from_pretrained: 사전 학습된 Object Detection 모델.

In [ ]:
# 인터넷 URL에서 이미지 로드
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# 로컬 파일에서 이미지 로드
# image_path = " "  # 이미지 경로를 여기에 입력하세요
# image = Image.open(image_path)

# 프로세서 및 모델 초기화
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

### 3. 모델 추론 및 결과값 반환
모델을 사용해 이미지를 추론하고, 그 결과를 처리하여 경계 상자와 레이블을 가져옵니다.

(참고) 함수설명,

- processor(images=image, return_tensors="pt"): 이미지를 모델에 맞게 전처리.
- model(**inputs): 모델을 사용해 추론 수행.
- torch.tensor([image.size[::-1]]): 모델이 출력할 대상 이미지의 크기를 지정.
- processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9): 모델의 결과를 경계 상자 및 레이블로 변환.

In [ ]:
# 이미지 전처리 및 추론 수행
inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# 출력값을 처리하여 경계 상자 및 레이블 얻기
target_sizes = torch.tensor([image.size[::-1]])  # 이미지 크기를 모델의 예상 결과 크기에 맞춤
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

#### 결과값 출력

In [ ]:
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )

### 4. 시각화
감지된 객체의 경계 상자와 레이블을 이미지 위에 시각화합니다.

(참고) 함수 설명,


- plt.subplots: 플롯을 그릴 수 있는 새 Figure 생성.
- ax.imshow(image): 이미지를 플롯에 추가.
- patches.Rectangle: 경계 상자를 그리기 위한 사각형 패치 생성.
- ax.add_patch(rect): 경계 상자 패치를 플롯에 추가.
- ax.text: 경계 상자 위에 레이블과 신뢰도를 표시.


In [ ]:
# 원본 이미지에 경계 상자를 오버레이하기 위한 시각화
fig, ax = plt.subplots(1, figsize=(12, 9))
ax.imshow(image)

# 각 경계 상자를 그리고 레이블 및 신뢰도 표시
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    label_name = model.config.id2label[label.item()]  # 레이블 이름 얻기
    confidence = round(score.item(), 3)  # 신뢰도
    rect = patches.Rectangle((box[0], box[1]), box[2] - box[0], box[3] - box[1], linewidth=2, edgecolor='r', facecolor='none')  # 경계 상자 그리기
    ax.add_patch(rect)  # 경계 상자 추가
    ax.text(box[0], box[1] - 10, f"{label_name}: {confidence}", color='red', fontsize=12, backgroundcolor="white")  # 레이블 및 신뢰도 표시

plt.axis('off')  # 축 숨기기
plt.show()  # 이미지 표시